In [28]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds
from copy import copy
from scipy.optimize import OptimizeResult, minimize
from optimparallel import minimize_parallel
from montecarlo import MonteCarlo
from scipy.optimize import differential_evolution

In [29]:
from scipy.stats import norm, nbinom
class ECIRModel:
    """Extended Cox-Ingersoll-Ross (CIR) model with Negative Binomial jumps for interest rate dynamics."""
    
    def __init__(self, kappa: float, mu_r: float, sigma: float, p: float, r: int, mu: float, gamma: float):
        """
        Initializes the extended CIR model with parameters.
        :param kappa: Mean reversion speed.
        :param mu_r: Long-term mean interest rate.
        :param sigma: Volatility of the interest rate.
        :param p: Probability of success in each Bernoulli trial for the Negative Binomial distribution.
        :param r: Number of successes until the process is stopped (Negative Binomial parameter).
        :param mu: Mean of the normal distribution for jump sizes.
        :param gamma: Standard deviation of the normal distribution for jump sizes.
        """
        self.kappa = kappa
        self.mu_r = mu_r
        self.sigma = sigma
        self.p = p
        self.r = r
        self.mu = mu
        self.gamma = gamma

    def next_rate(self, current_rate: float, dt: float) -> float:
        """
        Simulates the next interest rate using the Euler-Maruyama method.
        :param current_rate: Current interest rate.
        :param dt: Time increment.
        :return: Next interest rate.
        """
        normal_shock = np.random.normal(0, 1)
        drift = self.kappa * (self.mu_r - current_rate) * dt
        diffusion = self.sigma * np.sqrt(max(current_rate, 0)) * np.sqrt(dt) * normal_shock
        new_rate = current_rate + drift + diffusion
        return max(new_rate, 0)  # Ensure non-negativity

    def next_rate_with_jumps(self, current_rate: float, dt: float) -> float:
        """
        Simulates the next interest rate using the Euler-Maruyama method with Negative Binomial jumps.
        :param current_rate: Current interest rate.
        :param dt: Time increment.
        :return: Next interest rate including jumps.
        """
        # Standard CIR process simulation
        new_rate = self.next_rate(current_rate, dt)
        
        # Check for the occurrence of a jump
        num_jumps = np.random.negative_binomial(self.r, self.p)
        if num_jumps > 0:
            jump_sizes = np.random.normal(self.mu, self.gamma, num_jumps)
            total_jump = np.sum(jump_sizes)
            new_rate += total_jump
        
        return max(new_rate, 0)  # Ensure non-negativity

    def exact_solution(self, initial_rate: float, maturity: float) -> float:
        """
        Calculates the exact bond price under the CIR model for a zero-coupon bond.
        :param initial_rate: Initial interest rate.
        :param maturity: Time to maturity of the bond.
        :return: Bond price.
        """
        gamma = np.sqrt(self.kappa**2 + 2*self.sigma**2)
        B = (2 * (np.exp(gamma * maturity) - 1)) / ((gamma + self.kappa) * (np.exp(gamma * maturity) - 1) + 2 * gamma)
        A = ((2 * self.kappa * self.mu_r) / self.sigma**2) * np.log(
            2 * gamma * np.exp((gamma + self.kappa) * maturity / 2) / ((gamma + self.kappa) * (np.exp(gamma * maturity) - 1) + 2 * gamma))
        return np.exp(A - B * initial_rate)

    def transition_density(self, rt: float, rt_1: float, dt: float, terms_limit: int = 2) -> float:
        density_sum = 0
        for n in range(terms_limit):
            rate_increment = self.mu * n + self.kappa * (self.mu_r - rt_1) * dt
            variance = max(n * (self.gamma**2) + dt * (rt_1 * (self.sigma**2)), 1e-10)  # Ensure non-zero variance
            mean = rt_1 + rate_increment
            normal_density = norm.pdf(rt, loc=mean, scale=np.sqrt(variance))
            negative_binomial_density = nbinom.pmf(n, self.r, self.p)
            density_sum += normal_density * negative_binomial_density

        return density_sum


In [30]:
class Calibration:
    def __init__(self, model_class, n: int, m: int, r0: float, model_params: dict, optimize_args: tuple, calibrate_exact=False, seed=93756826):
        """
        Initialize the calibration class.
        """
        self.n = n  # Number of simulation paths
        self.m = m  # Time steps per simulation
        self.r0 = r0  # Initial interest rate
        self.seed = seed
        self.model_class = model_class  
        self.model_params = model_params  
        self.optimize_args = optimize_args  
        self.calibrate_exact = calibrate_exact  

    def _calculate_error(self, optimize_params: np.array, Ts: np.array, prices: np.array) -> float:
        model_params = copy(self.model_params)
        for arg, param in zip(self.optimize_args, optimize_params):
            model_params[arg] = float(param)  # Ensure appropriate type

        model = self.model_class(**model_params)
        mc = MonteCarlo(model, self.r0, 1, self.m, self.n)

        errors = []
        for T, market_price in zip(Ts, prices.flatten()):  # Flatten to ensure it's a 1D array
            simulated_price = mc.price_estimates()
            if isinstance(simulated_price, tuple):
                simulated_price = simulated_price[0]  # Assuming the first element is what you need
            error = (simulated_price - market_price) ** 2
            errors.append(error)

        return np.sqrt(np.mean(errors))


    def calibrate(self, bounds, Ts, prices):
        error_function = lambda params: self._calculate_error(params, Ts, prices)
        result = differential_evolution(
            error_function, 
            bounds=bounds, 
            maxiter=1000, 
            popsize=15, 
            tol=0.01, 
            mutation=(0.5, 1), 
            recombination=0.7,
            disp=True,
            strategy='best1bin'
        )
        return result

In [31]:
if __name__ == "__main__":
    # Load data and prepare
    df = pd.read_csv("DGS_30.csv", index_col=0)
    df.index = pd.to_datetime(df.index)
    df.index.name = 'DATE'

    selected_date = '2024-03-15'
    yields = df.loc[selected_date].astype(float) / 100
    
    yield_data = pd.DataFrame({
        'Yield': yields.values,
        'Maturity': np.arange(1, len(yields) + 1)
    })
    
    yield_data["Cum. Yield"] = yield_data["Yield"] * yield_data["Maturity"]
    yield_data["Price"] = np.exp(-yield_data["Cum. Yield"])
    
    prices = yield_data["Price"].values.reshape(-1, 1)
    Ts = yield_data["Maturity"].values
    
    initial_model_params = {
        "kappa": 0.5,
        "mu_r": 0.03,
        "sigma": 0.03,
        "mu": 0,
        "gamma": 0.01,
        "r": 10,
        "p": 0.5
    }
    
    bounds = [
    (0.001, 5),  # kappa
    (0.001, 0.1),  # mu_r
    (0.001, 0.5),  # sigma
    (0, 1),  # mu
    (0.001, 0.1),  # gamma
    (1, 100),  # r
    (0.001, 0.999)  # p
    ]

    calibrator = Calibration(
        model_class=ECIRModel, 
        n=100, m=26, r0=0.001,
        model_params=initial_model_params, 
        optimize_args=('kappa', 'mu_r', 'sigma', 'mu', 'gamma', 'r', 'p')
    )

    optimal_params = calibrator.calibrate(
        bounds=bounds,
        Ts=Ts,
        prices=prices
    )
    print(optimal_params)

differential_evolution step 1: f(x)= 0.4215309367958839
differential_evolution step 2: f(x)= 0.4215309367958839
differential_evolution step 3: f(x)= 0.4215298529579488
differential_evolution step 4: f(x)= 0.4215298529579488
differential_evolution step 5: f(x)= 0.4215298529579488
differential_evolution step 6: f(x)= 0.4215298529579488
differential_evolution step 7: f(x)= 0.4117413323868938
differential_evolution step 8: f(x)= 0.40561317184989215
Polishing solution with 'L-BFGS-B'
             message: Optimization terminated successfully.
             success: True
                 fun: 0.40561317184989215
                   x: [ 1.227e-02  9.854e-02  9.395e-02  2.995e-03
                        2.321e-03  6.579e+01  3.049e-01]
                 nit: 8
                nfev: 1161
          population: [[ 1.227e-02  9.854e-02 ...  6.579e+01  3.049e-01]
                       [ 2.883e+00  9.948e-02 ...  7.300e+01  6.921e-01]
                       ...
                       [ 1.809e+00  9.7

In [ ]:
params = {
    "kappa": 0.01227,    
    "mu_r": 0.09854,     
    "sigma": 0.09395,    
    "mu": 0.002995,      
    "gamma": 0.002321,   
    "r": 66,             
    "p": 0.3049          
}